In [1]:
import pyspark.sql.functions as F
import os
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkContext, SparkConf


#Spark session is being created
#Configuration of the Spark Session
app_name="instacart"
conf = SparkConf()  # create the configuration
conf.set('spark.driver.extraClassPath', "/usr/share/cmf/common_jars/mysql-connector-java-5.1.15.jar")  # set the spark.jars
conf.set('spark.executor.extraClassPath', "/usr/share/cmf/common_jars/mysql-connector-java-5.1.15.jar")

#Spark Session object
spark = SparkSession.builder.config(conf=conf).appName(app_name).getOrCreate()

In [2]:
#1. Load data into Spark DataFrame

orders = spark.read.csv("/user/edureka_960126/instacart/orders.csv",inferSchema=True,header=True)
products=spark.read.csv("/user/edureka_960126/instacart/products.csv",inferSchema=True,header=True)
departments=spark.read.csv("/user/edureka_960126/instacart/departments.csv",inferSchema=True,header=True)
aisles=spark.read.csv("/user/edureka_960126/instacart/aisles.csv",inferSchema=True,header=True)
order_products__prior=spark.read.csv("/user/edureka_960126/instacart/order_products__prior.csv",inferSchema=True,header=True)
order_products__train=spark.read.csv("/user/edureka_960126/instacart/order_products__train.csv",inferSchema=True,header=True)

orders.createOrReplaceTempView("orders")
products.createOrReplaceTempView("products")
departments.createOrReplaceTempView("departments")
aisles.createOrReplaceTempView("aisles")
order_products__prior.createOrReplaceTempView("order_products__prior")
order_products__train.createOrReplaceTempView("order_products__train")

#sql_df = spark.sql('Select * From orders')
#sql_df.show(10)
orders.show(10)
products.show(10)
departments.show(100)
aisles.show(100)
order_products__prior.show(10)
order_products__train.show(10)



+--------+-------+--------+------------+---------+-----------------+----------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|
+--------+-------+--------+------------+---------+-----------------+----------------------+
| 2539329|      1|   prior|           1|        2|                8|                  null|
| 2398795|      1|   prior|           2|        3|                7|                  15.0|
|  473747|      1|   prior|           3|        3|               12|                  21.0|
| 2254736|      1|   prior|           4|        4|                7|                  29.0|
|  431534|      1|   prior|           5|        4|               15|                  28.0|
| 3367565|      1|   prior|           6|        2|                7|                  19.0|
|  550135|      1|   prior|           7|        1|                9|                  20.0|
| 3108588|      1|   prior|           8|        1|               14|            

In [3]:
#2. Merge all the data frames based on the common key and create a single DataFrame 

#First products merged with aisles and departments

sdf=spark.sql('select o.order_id , o.user_id , o.eval_set , o.order_number ,o.order_dow ,o.order_hour_of_day ,o.days_since_prior_order , op.order_id , op.add_to_cart_order , op.reordered , p.product_id , p.product_name , p.aisle_id ,p.department_id , d.department , a.aisle from  orders o , order_products__prior op , products p , departments d, aisles a where o.order_id = op.order_id  and op.product_id = p.product_id and  p.department_id = d.department_id and p.aisle_id = a.aisle_id')

sdf.show(100)

+--------+-------+--------+------------+---------+-----------------+----------------------+--------+-----------------+---------+----------+--------------------+--------+-------------+---------------+--------------------+
|order_id|user_id|eval_set|order_number|order_dow|order_hour_of_day|days_since_prior_order|order_id|add_to_cart_order|reordered|product_id|        product_name|aisle_id|department_id|     department|               aisle|
+--------+-------+--------+------------+---------+-----------------+----------------------+--------+-----------------+---------+----------+--------------------+--------+-------------+---------------+--------------------+
|     148|  41523|   prior|          27|        2|               17|                   5.0|     148|                1|        0|     38650| Organic Red Lentils|      63|            9|dry goods pasta|grains rice dried...|
|     148|  41523|   prior|          27|        2|               17|                   5.0|     148|                

In [ ]:
#Queries for understanding of the Data

spark.sql('Select * From orders where user_id=202279').show(truncate=False)

spark.sql('Select * From order_products__prior where order_id in (Select order_id From orders where user_id=202279)').show(1000,truncate=False)

spark.sql('Select * From order_products__train where order_id in (Select order_id From orders where user_id=202279)').show(1000,truncate=False)

spark.sql('Select product_id From order_products__prior where order_id in (Select order_id From orders where user_id=202279) intersect Select product_id From order_products__train where order_id in (Select order_id From orders where user_id=202279)').show(1000,truncate=False)

In [7]:
#3.Check missing data
print(sdf.count())

print(order_products__prior.count())

#as from the results it can be soon that in our joining the tables we have diffrence of minute 3 records in such humungous dataset
# so our joining process is perfect



32434486
32434489


In [ ]:
#Note that I have used the individual tables to come out to the results of questions asked , combined sdf table can also be used

# as in the assignment our task was to come out with the results for the question asked not explicitly mentioned to use the combined table

In [8]:
#4.List the most ordered products (top 10)

#Here using the order_products_prior to calculate the count of the products in various orders and then sort them and get 10

spark.sql('Select product_id , count(product_id) from order_products__prior group by product_id  order by count(product_id)  desc limit 10').show()

spark.sql(' Select product_id , product_name from products where product_id in (Select product_id  from order_products__prior group by product_id  order by count(product_id)  desc limit 10)').show(1000)

spark.sql(' select a.product_id ,a.counts, p.product_name from (Select product_id , count(product_id) as counts from order_products__prior group by product_id  order by count(product_id)  desc limit 10) as a inner join products as p where a.product_id = p.product_id ').show(1000)



+----------+-----------------+
|product_id|count(product_id)|
+----------+-----------------+
|     24852|           472565|
|     13176|           379450|
|     21137|           264683|
|     21903|           241921|
|     47209|           213584|
|     47766|           176815|
|     47626|           152657|
|     16797|           142951|
|     26209|           140627|
|     27845|           137905|
+----------+-----------------+

+----------+--------------------+
|product_id|        product_name|
+----------+--------------------+
|     13176|Bag of Organic Ba...|
|     16797|        Strawberries|
|     21137|Organic Strawberries|
|     21903|Organic Baby Spinach|
|     24852|              Banana|
|     26209|               Limes|
|     27845|  Organic Whole Milk|
|     47209|Organic Hass Avocado|
|     47626|         Large Lemon|
|     47766|     Organic Avocado|
+----------+--------------------+

+----------+------+--------------------+
|product_id|counts|        product_name|
+-----

In [9]:
#5. Do people usually reorder the same previous ordered products? 

#This can be solved like we can have product wise ordered numbers , reordered numbers , then reorderd/ordered will give us the reordering probability
#reordering probability of each product
#then we can take mean of this reordering probability to get the overall attitude of people to reorder

spark.sql('Select product_id ,count(product_id) as purchase_count , sum(reordered) as reorderd_count , sum(reordered)/count(product_id) as reordering_probability  from order_products__prior group by product_id  order by reordering_probability desc').show()

#Overall mean of the reordering probability will decide the overall attitude of public in reordering

spark.sql('Select avg(a.reordering_probability) as reordering_sentiment from (Select product_id ,count(product_id) as purchase_count , sum(reordered) as reorderd_count , sum(reordered)/count(product_id) as reordering_probability  from order_products__prior group by product_id  order by reordering_probability desc) as a').show()


+----------+--------------+--------------+----------------------+
|product_id|purchase_count|reorderd_count|reordering_probability|
+----------+--------------+--------------+----------------------+
|      6433|            68|            64|    0.9411764705882353|
|      2075|            87|            81|    0.9310344827586207|
|     43553|            13|            12|    0.9230769230769231|
|     27740|           101|            93|    0.9207920792079208|
|     14609|            35|            32|    0.9142857142857143|
|     13875|            45|            41|    0.9111111111111111|
|     39992|            22|            20|    0.9090909090909091|
|      5868|            30|            27|                   0.9|
|     35604|           100|            90|                   0.9|
|     31418|            60|            54|                   0.9|
|     36543|            67|            60|    0.8955223880597015|
|     26093|            66|            59|    0.8939393939393939|
|       70

In [ ]:
#As we can see above the average of reordering probability is 0.37 i.e. around 37 percent so it seems that people usually not reorder the products

In [10]:
#6. List most reordered products 

# here we sum the reordering value with respect to product_id and then sort in descending order 

spark.sql('Select product_id , sum(reordered) from order_products__prior group by product_id  order by sum(reordered)  desc limit 10').show()


spark.sql(' select a.product_id ,a.reordering_count, p.product_name from (Select product_id , sum(reordered) as reordering_count from order_products__prior group by product_id  order by sum(reordered)  desc limit 10) as a inner join products as p where a.product_id = p.product_id ').show(1000)



+----------+--------------+
|product_id|sum(reordered)|
+----------+--------------+
|     24852|        398609|
|     13176|        315913|
|     21137|        205845|
|     21903|        186884|
|     47209|        170131|
|     47766|        134044|
|     27845|        114510|
|     47626|        106255|
|     27966|        105409|
|     16797|         99802|
+----------+--------------+

+----------+----------------+--------------------+
|product_id|reordering_count|        product_name|
+----------+----------------+--------------------+
|     24852|          398609|              Banana|
|     13176|          315913|Bag of Organic Ba...|
|     21137|          205845|Organic Strawberries|
|     21903|          186884|Organic Baby Spinach|
|     47209|          170131|Organic Hass Avocado|
|     47766|          134044|     Organic Avocado|
|     27845|          114510|  Organic Whole Milk|
|     47626|          106255|         Large Lemon|
|     27966|          105409| Organic Raspberr

In [11]:
#7. Most important department and aisle (by number of products)

spark.sql(' Select department_id , department from departments where department_id in (Select department_id  from products group by department_id  order by count(product_id)  desc limit 1)').show(1000)


#spark.sql(' Select aisle_id  from products group by aisle_id  order by count(product_id)  desc limit 1').show(1000)

spark.sql(' Select aisle_id , aisle from aisles where aisle_id in (Select aisle_id  from products group by aisle_id  order by count(product_id)  desc limit 1)').show(1000)


+-------------+-------------+
|department_id|   department|
+-------------+-------------+
|           11|personal care|
+-------------+-------------+

+--------+-------+
|aisle_id|  aisle|
+--------+-------+
|     100|missing|
+--------+-------+



In [12]:
#8. Get the Top 10 departments

#So here the number of products are the deciding factor right

#Through this query we will get the top 10 departments with their names but they will not be in order according to the products
spark.sql(' Select department_id , department from departments where department_id in (Select department_id  from products group by department_id  order by count(product_id)  desc limit 10)').show(1000)


#Here we have the top 10 departments based on the count of products 
spark.sql('Select department_id  , count(product_id) as product_count from products group by department_id  order by count(product_id)  desc limit 10').show()


#Using both queries to make complex query to generate the results
spark.sql('Select d.department_id , d.department , a.product_count from (Select department_id  , count(product_id) as product_count from products group by department_id  order by count(product_id)  desc limit 10) as a inner join departments d where a.department_id = d.department_id ').show()




+-------------+---------------+
|department_id|     department|
+-------------+---------------+
|            1|         frozen|
|            4|        produce|
|            7|      beverages|
|            9|dry goods pasta|
|           11|  personal care|
|           13|         pantry|
|           15|   canned goods|
|           16|     dairy eggs|
|           17|      household|
|           19|         snacks|
+-------------+---------------+

+-------------+-------------+
|department_id|product_count|
+-------------+-------------+
|           11|         6563|
|           19|         6264|
|           13|         5371|
|            7|         4365|
|            1|         4007|
|           16|         3449|
|           17|         3084|
|           15|         2092|
|            9|         1858|
|            4|         1684|
+-------------+-------------+

+-------------+---------------+-------------+
|department_id|     department|product_count|
+-------------+---------------+-------

In [13]:
#9. List top 10 products ordered in the morning (6 AM to 11 AM)

spark.sql('Select product_id , count(product_id) from order_products__prior  where order_id in (select distinct order_id from orders where (order_hour_of_day >= 6 and order_hour_of_day <= 11 )) group by product_id  order by count(product_id)  desc limit 10 ').show()

spark.sql(' select a.product_id ,a.order_counts, p.product_name from (Select product_id , count(product_id) as order_counts from order_products__prior  where order_id in (select distinct order_id from orders where (order_hour_of_day >= 6 and order_hour_of_day <= 11 )) group by product_id  order by count(product_id)  desc limit 10 ) as a inner join products as p where a.product_id = p.product_id ').show(1000)





+----------+-----------------+
|product_id|count(product_id)|
+----------+-----------------+
|     24852|           163891|
|     13176|           130488|
|     21137|            89015|
|     21903|            79550|
|     47209|            70312|
|     47766|            57311|
|     47626|            50905|
|     16797|            49982|
|     27845|            48124|
|     27966|            47948|
+----------+-----------------+

+----------+------------+--------------------+
|product_id|order_counts|        product_name|
+----------+------------+--------------------+
|     24852|      163891|              Banana|
|     13176|      130488|Bag of Organic Ba...|
|     21137|       89015|Organic Strawberries|
|     21903|       79550|Organic Baby Spinach|
|     47209|       70312|Organic Hass Avocado|
|     47766|       57311|     Organic Avocado|
|     47626|       50905|         Large Lemon|
|     16797|       49982|        Strawberries|
|     27845|       48124|  Organic Whole Milk|
|

In [15]:
#testing queries to confirm the above results

#spark.sql('Select  order_id from order_products__prior where product_id = 24852').show(10000)
#spark.sql('(Select  order_id from order_products__prior where product_id = 24852) intersect (Select order_id  from orders where order_id in ( Select  order_id from order_products__prior where product_id = 24852) and order_hour_of_day between 6 and 11)').show(10000)
#spark.sql('select order_hour_of_day from orders where order_id=1537').show()
#spark.sql('select product_id from order_products__prior where order_id=1537').show()

In [16]:
#10. Create a spark-submit application for the same and print the findings in the log


# I have submitted the Jupyter notebook instead of creating the pyspark script and then submitting the pyspark job 